### Connect to Milvus Server

In [1]:
from pymilvus import connections
connections.connect(
  alias="default",
  uri="http://localhost:19530",
)

### Create a Collection

In [2]:
from pymilvus import CollectionSchema, FieldSchema, DataType
book_id = FieldSchema(
  name="book_id",
  dtype=DataType.INT64,
  is_primary=True,
)
book_name = FieldSchema(
  name="book_name",
  dtype=DataType.VARCHAR,
  max_length=200,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)
word_count = FieldSchema(
  name="word_count",
  dtype=DataType.INT64,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value=9999
)
book_intro = FieldSchema(
  name="book_intro",
  dtype=DataType.FLOAT_VECTOR,
  dim=2
)
schema = CollectionSchema(
  fields=[book_id, book_name, word_count, book_intro],
  description="Test book search",
  enable_dynamic_field=True
)
collection_name = "book"



In [10]:
from pymilvus import Collection
collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2
    )

### Insert Data into Milvus



In [11]:
import random
data = [
  [i for i in range(2000)],
  [str(i) for i in range(2000)],
  [i for i in range(10000, 12000)],
  [[random.random() for _ in range(2)] for _ in range(2000)],
  # use `default_value` for a field
  [], 
  # or
  None,
  # or just omit the field
]

data.append([str("dy"*i) for i in range(2000)])


In [13]:
from pymilvus import Collection
collection = Collection(collection_name)      # Get an existing collection.
mr = collection.insert(data[:4])  # Insert only the first 4 lists that match the schema fields.


### Build an index on vectors


In [14]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}


In [15]:
from pymilvus import Collection, utility
collection = Collection("book")      
collection.create_index(
  field_name="book_intro", 
  index_params=index_params
)

utility.index_building_progress("book")


{'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0}

### Vector Similarity search


In [22]:
collection.load()

In [17]:
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}


In [18]:
results = collection.search(
    data=[[0.1, 0.2]], 
    anns_field="book_intro", 
    # the sum of `offset` in `param` and `limit` 
    # should be less than 16384.
    param=search_params,
    limit=10,
    expr=None,
    # set the names of the fields you want to 
    # retrieve from the search result.
    output_fields=['title'],
    consistency_level="Strong"
)

results[0].ids

results[0].distances

hit = results[0][0]
hit.entity.get('title')


In [19]:
results[0].ids
results[0].distances


[0.00017925072461366653,
 0.00042345491237938404,
 0.0007979392539709806,
 0.0010629157768562436,
 0.0011057660449296236,
 0.0014931642217561603,
 0.0015233766753226519,
 0.0015477623092010617,
 0.0015534862177446485,
 0.0017442794051021338]

### Query Vector

In [23]:
res = collection.query(
  expr = "book_id in [2,4,6,8]",
  offset = 0,
  limit = 10, 
  output_fields = ["book_id", "book_intro"],
)


In [24]:
sorted_res = sorted(res, key=lambda k: k['book_id'])
sorted_res


[{'book_intro': [0.9614506, 0.67399], 'book_id': 2},
 {'book_intro': [0.6594746, 0.74511784], 'book_id': 4},
 {'book_intro': [0.07552183, 0.86942947], 'book_id': 6},
 {'book_intro': [0.3202323, 0.042471204], 'book_id': 8}]

In [ ]:
# Disconnect from Milvus server
connections.disconnect("default")